In [ ]:
using AcausalNets
using QI
using LightGraphs
using LinearAlgebra

In [ ]:
roA = diagm(0 =>[
        .5, #A0
        .5  #A1
        ])

roBwA = diagm(0 => [
        .6, #A0, B0
        .4, #A0, B1
        .5, #A1, B0
        .5, #A1, B1
        ])

roCwA = diagm(0 => [
        .8, #A0, C0
        .2, #A0, C1
        .3, #A1, C0
        .7  #A1, C1
        ])

roDwB = diagm(0 => [
        .5, #B0, D0
        .5, #B0, D1
        .1, #B1, D0
        .9  #B1, D1
        ])

roEwC = diagm(0 => [
        .4, #C0, E0
        .6, #C0, E1
        .7, #C1, E0
        .3  #C1, E1
        ])

roFwDE = diagm(0 => [
        .01, #D0, E0, F0
        .99, #D0, E0, F1
        .99, #D0, E1, F0
        .01, #D0, E1, F1
        .99, #D1, E0, F0
        .01, #D1, E0, F1
        .99, #D1, E1, F0
        .01  #D1, E1, F1
        ])

roGwC = diagm(0 => [
        .9, #C0, G0
        .1, #C0, G1
        .2, #C1, G0
        .8  #C1, G1
        ])
roHwEG = diagm(0 => [
        .05, #E0, G0, H0
        .95, #E0, G0, H1
        .05, #E0, G1, H0
        .95, #E0, G1, H1
        .05, #E1, G0, H0
        .95, #E1, G0, H1
        .95, #E1, G1, H0
        .05  #E1, G1, H1
        ])

var_a = Variable(:a, 2)
var_b = Variable(:b, 2)
var_c = Variable(:c, 2)
var_d = Variable(:d, 2)
var_e = Variable(:e, 2)
var_f = Variable(:f, 2)
var_g = Variable(:g, 2)
var_h = Variable(:h, 2)

sys_a = DiscreteQuantumSystem([var_a], roA)
sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
sys_c = DiscreteQuantumSystem([var_a], [var_c], roCwA)
sys_d = DiscreteQuantumSystem([var_b], [var_d], roDwB)
sys_e = DiscreteQuantumSystem([var_c], [var_e], roEwC)
sys_f = DiscreteQuantumSystem([var_d, var_e], [var_f], roFwDE)
sys_g = DiscreteQuantumSystem([var_c], [var_g], roGwC)
sys_h = DiscreteQuantumSystem([var_e, var_g], [var_h], roHwEG)


example_an = AcausalNet()
push!(example_an, sys_a)
push!(example_an, sys_b)
push!(example_an, sys_c)
push!(example_an, sys_d)
push!(example_an, sys_e)
push!(example_an, sys_f)
push!(example_an, sys_g)
push!(example_an, sys_h)

an = example_an
show(an, true)

In [ ]:
a_obs = Evidence([var_a], ketbra(1,1,2))

observations = Evidence{Matrix{Float64}}[] 
observations = [a_obs]

In [ ]:
to_infer = Variable[var_g, var_b, var_e] #, var_b]
# inferred_system_belief, _ = infer_belief_debug(inferrer, to_infer, observations)
inferred_system, debug = infer_join_tree(an, to_infer, observations)
inferred_system_naive, debug_naive = infer_naive(an, to_infer, observations);
inferred_system_belief, debug_beleif = infer_belief(an, to_infer, observations);

In [ ]:
(dbn,mg,enforced_mg,tri_mg,cliques,
        parent_cliques,initialized_jt,observations_jt,propagated_jt,jt,) = debug
(dbn,cluster,obs,evidence_cluster,inferred_cluster) = debug_naive
show(jt);

In [ ]:
real(distribution(inferred_system))

In [ ]:
real(distribution(inferred_system_naive))

In [ ]:
real(distribution(inferred_system_belief))

In [ ]:
[[[v.name for v in variables(sys)] for sys in cl] for cl in cliques]

In [ ]:
show(jt)

In [ ]:
[Set([src(e), dst(e)]) for e in edges(jt.graph)]